In [ ]:
!pip install dspy

In [ ]:
import pandas as pd
import openai
import dspy

In [ ]:
from google.colab import userdata
OPENAI_KEY = userdata.get('GPT_KEY')
openai.api_key = OPENAI_KEY

In [ ]:
openai_model = dspy.OpenAI(model='gpt-3.5-turbo',api_key = OPENAI_KEY)
dspy.settings.configure(lm=openai_model)

In [ ]:
data = pd.read_csv("/content/ADO_FeatureUserStorylist_V2.xlsx - Sheet1.csv", header = None)
data.head()

In [ ]:
data.columns = data.iloc[1]
data = data.drop([0, 1]).reset_index(drop=True)
data.head()

In [ ]:
example_data = {
    "Work Item Type": ["Feature", "User Story"],
    "Created Date": ["2/2/2023", "1/4/2024"],
    "Feature Title": ["Compensation rates are not populating in Concessions reports.", None],
    "User Story Title": [None, "Finance Source - Develop to pull compensation rates"],
    "Assigned To": ["Anthony", "Anthony"],
    "State": ["Active", "On Hold"],
    "AskIT Enhancement Number": ["ENHC0014455", "ENHC0014455"],
    "Customer Name": ["Branden Flowers", "Poppy"],
    "Iteration Path": [None, None],
    "Area Path": [None, None],
    "Level of Effort": ["Medium", "Low"],
    "Story Points (0 - 99)": [40, 20],
    "Value (0 - 99)": [None, None],
    "Priority (0 - 4)": [2, 1],
    "Target Date": [None, None],
    "Business Value (0 - 999)": [450, 150]
}

# Convert the dictionary into a DataFrame
example_df = pd.DataFrame(example_data)

example_df.head()

In [ ]:
class StoryPointsAgent(dspy.Signature):
    example_data = dspy.InputField(desc="Example input data")

    f"""Analyze 'State', 'Feature Title', and 'User Story Title' to assign Story Points based on task complexity.

    Examples: { example_data }

    From the example learn how to assign story points.
    A relative scale for story points to estimate task complexity. We'll enforce the range and handle edge cases where tasks might exceed the expected limits.
    """
    dataset = dspy.InputField()
    context = dspy.InputField(desc="Context: Story points reflect task complexity and estimated completion time.")
    result = dspy.OutputField(desc="Output value: {<Between 0-99>}")

class EffortAgent(dspy.Signature):
    example_data = dspy.InputField(desc="Example input data")

    f"""Analyze 'State', 'Feature Title', and 'User Story Title' to assign Level of Effort.

    Examples: { example_data }

    From the example learn how to assign Level of Effort.
    Constraints will depend on the complexity of the task but should fall within a reasonable range assign Low, Medium or High.
    Features with high complexity needs higher efforts, while smaller tasks (like bug fixes or simple enhancements) would get lower values.
    """
    dataset = dspy.InputField()
    context = dspy.InputField(desc="Context: Level of Effort reflects the estimated amount of work required.")
    result = dspy.OutputField(desc="Output value: {<Low,Medium or High>}")

class PriorityAgent(dspy.Signature):
    example_data = dspy.InputField(desc="Example input data")

    f"""Analyze 'State', 'Feature Title', and 'User Story Title' to assign task Priority.

    Examples: { example_data }

    From the example learn how to assign task Priority.
    3-4 for critical tasks or active work.
    1-2 for new work that isn't immediately blocking.
    0-1 for tasks that are On Hold or Closed.
    """
    dataset = dspy.InputField()
    context = dspy.InputField(desc="Context: Priority is based on task urgency and impact.")
    result = dspy.OutputField(desc="Output value: {<Between 0-4>}")

class BusinessValueAgent(dspy.Signature):
    example_data = dspy.InputField(desc="Example input data")

    f"""Analyze 'State', 'Feature Title', and 'User Story Title' to assign Business Value.

    Examples: { example_data }

    From the example learn how to assign Business Value.
    High values for tasks that directly impact customer experience, revenue, or core product features.
    Medium values for important but less critical tasks.
    Low values for non-essential or nice-to-have tasks.

    """
    dataset = dspy.InputField()
    context = dspy.InputField(desc="Context: Business Value reflects the strategic importance of the task.")
    result = dspy.OutputField(desc="Output value: {<Between 0-999>}")

class CombineAgent(dspy.Signature):
    """Combine results from each agent to create a unified filled record."""
    story_points_result = dspy.InputField(desc="Output from Story Points agent")
    effort_result = dspy.InputField(desc="Output from Level of Effort agent")
    priority_result = dspy.InputField(desc="Output from Priority agent")
    business_value_result = dspy.InputField(desc="Output from Business Value agent")
    final_result = dspy.OutputField(desc="output the format like story_points: {<only value>}, effort: {<only value>}, Priority: {<only value}, Business_value: {<only value}")

# Define the main DSPy workflow
class TaskImputationWorkflow(dspy.Module):
    def __init__(self, agents):
        # Initialize agents for each imputation task
        self.agents = {a.__pydantic_core_schema__['schema']['model_name']: dspy.ChainOfThought(a) for a in agents}
        self.combine_agent = dspy.ChainOfThought(CombineAgent)

    def forward(self, dataset, context, example_data):
        output_dict = {}

        # Iterate through each agent and fill missing values
        for name, agent in self.agents.items():
            result = agent(dataset=dataset.to_string(), context=context[name], example_data = example_data.to_string())
            output_dict[name] = result.result

        # Combine agent outputs
        combined_result = self.combine_agent(
            story_points_result=output_dict['StoryPointsAgent'],
            effort_result=output_dict['EffortAgent'],
            priority_result=output_dict['PriorityAgent'],
            business_value_result=output_dict['BusinessValueAgent']
        )

        return combined_result.final_result, output_dict

# Initialize agents for imputation workflow
agents = [StoryPointsAgent, EffortAgent, PriorityAgent, BusinessValueAgent]
task_imputation_workflow = TaskImputationWorkflow(agents)

# Define context with domain-specific logic
context_data = {
    "StoryPointsAgent": "Use 'State', 'Feature Title', and 'User Story Title' to estimate the complexity and duration for assigning story points.",
    "EffortAgent": "Estimate Level of Effort based on task descriptions and complexity indicated by 'Feature Title' and 'User Story Title'.",
    "PriorityAgent": "Assign Priority based on urgency and importance inferred from 'State' and task descriptions.",
    "BusinessValueAgent": "Calculate Business Value based on the strategic impact and alignment with project goals."
}

# Process each row in the dataset
results = []
for _, row in data.iterrows():
    dataset = pd.DataFrame([row])  # Convert row to DataFrame for DSPy agent
    filled_values, output_dict = task_imputation_workflow.forward(dataset, context_data, example_df)
    results.append(filled_values)


# # Save results to a new DataFrame and export
# df_filled = pd.DataFrame(results, columns=df.columns)
# df_filled.head()
# # df_filled.to_csv('/mnt/data/ADO_FeatureUserStorylist_V2_Filled.csv', index=False)


 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


In [ ]:
for item in results:
  print(item)

In [ ]:
import random

In [ ]:
output = []
for item in results:
  new = item.split(",")
  if len(new) !=4:
    random_list = str('Priority: '+ str(random.choice([0, 1])))
    new = list(new[:2]) + [random_list] + list(new[-1:])
  new_value = []
  for j in range(len(new)):
    try:
      value = new[j].split(": ")[1]
      new_value.append(value)
    except IndexError:
      new_value.append(None)
  output.append(new_value)

In [ ]:
data[['Story Points (0 - 99)', 'Level of Effort', 'Priority (0 - 4)', 'Business Value (0 - 999)']] = pd.DataFrame(output)
data.head()

In [ ]:
data.to_csv('/content/ADO_FeatureUserStorylist_V2_Filled.csv', index=False)